In [12]:
%pip install rouge-score

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached absl_py-2.2.1-py3-none-any.whl.metadata (2.4 kB)
Using cached absl_py-2.2.1-py3-none-any.whl (277 kB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=25025 sha256=57898b381bb126f5f86cdd75bc0520b03128cc525fa86560c14df54d10d1b99c
  Stored in directory: c:\users\limyi\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [7]:
# Combine 5 batches into 1
batches = ["./data/keywords_without_icl/first_batch.csv",
           "./data/keywords_without_icl/second_batch.csv",
           "./data/keywords_without_icl/third_batch.csv",
           "./data/keywords_without_icl/fourth_batch.csv",
           "./data/keywords_without_icl/last_batch.csv"]
combined_df = pd.DataFrame()

for batch in batches:
    df = pd.read_csv(batch)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

print("Number of rows in combined df:", len(combined_df))

combined_df.to_csv("./data/keywords_without_icl/test_dataset_keywords.csv", index=False)

Number of rows in combined df: 2296


In [ ]:
# Append statement column to keywords
test_df = pd.read_csv("./data/liar_test.csv")
keywords_df = pd.read_csv("./data/keywords_without_icl/test_dataset_keywords.csv")

keywords_df["statement"] = test_df["statement"]
keywords_df.to_csv("./data/keywords_without_icl/test_dataset_keywords.csv", index=False)

In [ ]:
# Clean keywords dataset to ignore statements with "UNKNOWN" label generated by in-context learning

# For zero-shot
keywords_df = pd.read_csv("./data/keywords_without_icl/test_dataset_keywords.csv")
zero_shot_df = pd.read_csv("./data/results_zero-shot_v2.csv")
filtered_df = keywords_df[keywords_df["statement"].isin(zero_shot_df["Claim"])]
filtered_df.to_csv("./data/keywords_without_icl/filtered_for_zero_shot.csv", index=False)

# For few-shot
keywords_df = pd.read_csv("./data/keywords_without_icl/test_dataset_keywords.csv")
few_shot_df = pd.read_csv("./data/results_few-shot_v2.csv")
filtered_df = keywords_df[keywords_df["statement"].isin(few_shot_df["Claim"])]
filtered_df.to_csv("./data/keywords_without_icl/filtered_for_few_shot.csv", index=False)

Number of matching statements: 2289
Filtered zero-shot rows: 2289
Zero-shot DataFrame rows: 2289
Missing values in zero_shot_df: 1


C:\Users\limyi\AppData\Local\Temp\ipykernel_18600\1381752769.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={"keywords": "keywords_from_roberta"}, inplace=True)
C:\Users\limyi\AppData\Local\Temp\ipykernel_18600\1381752769.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["keywords_from_icl"] = zero_shot_df["Keywords"]
C:\Users\limyi\AppData\Local\Temp\ipykernel_18600\1381752769.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

PermissionError: [Errno 13] Permission denied: './data/keywords_without_icl/filtered_for_few_shot.csv'

Rouge-1 Metrics

R: The cat is on the mat.

C: The cat and the dog.

ROUGE-1 precision can be computed as the ratio of the number of unigrams in C that also appears in R, over the number of unigrams in C. ROUGE-1 precision = 3/5

ROUGE-1 recall can be computed as the ratio of the number of unigrams in R that also appear in C, over the number of unigrams in R. ROUGE-1 recall = 3/6

We will be using ROUGE-1 recall as our metrics for comparison, where R is the keywords generated by our binary ROBERTA classifier, and C is the key phrases generated by in-context learning.

In [21]:
# Pre-condition: zero/few_shot_df needs to have columns: keywords_from_icl, keywords_from_roberta

import pandas as pd
from rouge_score import rouge_scorer

zero_shot_df = pd.read_csv("./data/keywords_without_icl/filtered_for_zero_shot.csv")
few_shot_df = pd.read_csv("./data/keywords_without_icl/filtered_for_few_shot.csv")

zero_shot_df["keywords_from_icl"] = zero_shot_df["keywords_from_icl"].fillna("").astype(str)
zero_shot_df["keywords_from_roberta"] = zero_shot_df["keywords_from_roberta"].fillna("").astype(str)
few_shot_df["keywords_from_icl"] = few_shot_df["keywords_from_icl"].fillna("").astype(str)
few_shot_df["keywords_from_roberta"] = few_shot_df["keywords_from_roberta"].fillna("").astype(str)

zero_shot_icl_keywords = zero_shot_df["keywords_from_icl"]
zero_shot_roberta_keywords = zero_shot_df["keywords_from_roberta"]
few_shot_icl_keywords = few_shot_df["keywords_from_icl"]
few_shot_roberta_keywords = few_shot_df["keywords_from_roberta"]

scorer = rouge_scorer.RougeScorer(['rouge1'])

rouge_scores_zero_shot = []
# roberta is Reference, icl is Candidate
for roberta_keywords, icl_keywords in zip(zero_shot_roberta_keywords, zero_shot_icl_keywords):
    scores = scorer.score(roberta_keywords, icl_keywords)
    rouge_scores_zero_shot.append(scores)

rouge_scores_few_shot = []
# roberta is Reference, icl is Candidate
for roberta_keywords, icl_keywords in zip(few_shot_roberta_keywords, few_shot_icl_keywords):
    scores = scorer.score(roberta_keywords, icl_keywords)
    rouge_scores_few_shot.append(scores)


zero_shot_df["rouge_scores"] = rouge_scores_zero_shot
zero_shot_df["rouge_recall"] = zero_shot_df["rouge_scores"].apply(lambda x: x["rouge1"].recall)
zero_shot_df.to_csv("./data/keywords_without_icl/zero_shot_rouge.csv")

few_shot_df["rouge_scores"] = rouge_scores_few_shot
few_shot_df["rouge_recall"] = few_shot_df["rouge_scores"].apply(lambda x: x["rouge1"].recall)
few_shot_df.to_csv("./data/keywords_without_icl/few_shot_rouge.csv")
